# Data Wangling for 2018 LC Loan Datasets
Perform data wrangling for 2018 loan data collected from LendingClub. The datasets include four csv files containing loan data for each quarter in 2018. The datasets can be founded [here](https://github.com/nphan20181/Loan-Default-Prediction/tree/master/data).

## Data Wrangling Approach

### Removing Columns
Remove columns that contain information that is not useful or readily available at the time a loan is issued. Examples of those columns are loan id, hardship flag, total received interest, etc. There are 98 columns that are subjects of interest such as loan amount, interest rate, etc. 

### Removing Rows
Remove all records that do not have a loan status of fully paid, charged off or default. 

### Missing Values
Remove columns that have more than 25% of missing values. There are 18 columns. The missing-value percentage for those columns ranges from 54% to 96%. Since the missing-value percentage for those columns is high, it is necessary to remove the columns. 

For columns that have less than or equal to 25% of missing values, fill in the median value for numerical variables and leave the values as null for non-numerical variables. There are 2 non-numerical columns and 13 numerical columns. The missing-value percentage for those columns ranges from 0.01% to 17%. 

### Outliers
Compute z-scores to obtain the records and variables that contain outliers (z-score < -3 or z-score > 3). There are 36,355 rows and 64 columns that contain outliers. Since the number of outliers in the dataset is high (195,492 / 495,242 = 39%), it is necessary to keep the outliers because they may contain significant information, and there are also some models that work well with outliers.

### New Columns
#### Loan Status Flag
Categorize loan status into 2 categories:
 - Fully paid
 - Default (Default, Charged Off)

#### Return on Investment (ROI) 
- ROI = total payment amount – loan amount

#### Months on Loan 
- Months on Loan = last payment date – loan issued date

### Export Data
- Export data to a pickle file for later use.
- Export data dictionary for variables of interest.

<hr>

In [1]:
import modules.global_vars as gv            # load user-defined variables
import glob
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 150)   # show max 150 columns

## Step 1: Import Data

In [2]:
zip_files = glob.glob('data/*.zip')   # get a list of zip files
list_data = []

for filename in zip_files:
    # read csv file into data frame, skip first row which contains general note
    data = pd.read_csv(filename, skiprows=[0], low_memory=False, 
                      usecols=['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
                               'home_ownership', 'annual_inc', 'issue_d', 'loan_status',
                               'title', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 
                               'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec',
                               'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'total_pymnt',
                               'last_pymnt_d', 'collections_12_mths_ex_med',
                               'mths_since_last_major_derog', 'application_type', 'annual_inc_joint', 'dti_joint',
                               'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il',
                               'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
                               'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim',
                               'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
                               'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
                               'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
                               'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
                               'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd',
                               'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
                               'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
                               'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
                               'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
                               'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
                               'revol_bal_joint', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
                               'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il',
                               'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
                               'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog'])

    # drop last 2 rows that contain the total amount funded in policy code 1 and 2
    data.drop(axis=0, index=data.iloc[-2:,:].index, inplace=True)
    list_data.append(data)

# combine into one data frame and drop column 'index'
df = pd.concat(list_data).reset_index().drop(axis=1, columns=['index'])

# keep only loans with status Fully Paid / Charged Off / Default
df = pd.concat([df.loc[df['loan_status'] == 'Fully Paid'],
          df.loc[df['loan_status'] == 'Charged Off'],
          df.loc[df['loan_status'] == 'Default']]).reset_index().drop(axis=1, columns=['index'])

## Step 2: Inspect Data
- [Date Columns](#Date-Columns)
- [Numerical Columns](#Numerical-Columns)
- [Non-numerical Columns](#Non-numerical-Columns)
- [Duplicates](#Duplicates)

In [3]:
# show number of rows and columns
df.shape

(93853, 96)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93853 entries, 0 to 93852
Data columns (total 96 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            93853 non-null  float64
 1   term                                 93853 non-null  object 
 2   int_rate                             93853 non-null  object 
 3   installment                          93853 non-null  float64
 4   grade                                93853 non-null  object 
 5   emp_length                           85734 non-null  object 
 6   home_ownership                       93853 non-null  object 
 7   annual_inc                           93853 non-null  float64
 8   issue_d                              93853 non-null  object 
 9   loan_status                          93853 non-null  object 
 10  title                                93853 non-null  object 
 11  dti                         

### Date Columns

In [5]:
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d']

# convert to date type
for d in date_cols:
    df[d] = pd.to_datetime(df[d])

df[date_cols].head()

,issue_d,earliest_cr_line,last_pymnt_d
0,2018-03-01,1998-01-01,2018-05-01
1,2018-03-01,2001-08-01,2018-04-01
2,2018-03-01,2007-03-01,2019-03-01
3,2018-03-01,2007-10-01,2019-04-01
4,2018-03-01,2002-03-01,2019-05-01


### Numerical Columns

In [6]:
num_cols_df = df.select_dtypes(include=["number"])     # numerical columns
num_cols_df.columns

Index(['loan_amnt', 'installment', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp',
       'total_pymnt', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',

In [7]:
num_cols_df.head()

,loan_amnt,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,3000.0,93.10,52000.0,0.58,0.0,0.0,26.0,NaN,7.0,0.0,141.0,30.0,0.0,3011.577285,0.0,NaN,NaN,NaN,0.0,0.0,150592.0,0.0,0.0,1.0,2.0,7.0,0.0,NaN,0.0,1.0,141.0,1.0,31000.0,1.0,2.0,2.0,3.0,25099.0,30359.0,0.5,0.0,0.0,132.0,242.0,18.0,7.0,4.0,18.0,NaN,7.0,NaN,0.0,1.0,1.0,4.0,15.0,7.0,6.0,19.0,1.0,7.0,0.0,0.0,0.0,1.0,96.7,0.0,0.0,0.0,191216.0,141.0,30500.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5000.0,166.03,55000.0,14.18,0.0,0.0,74.0,82.0,14.0,1.0,11449.0,24.0,0.0,5013.306667,0.0,74.0,NaN,NaN,0.0,0.0,28880.0,1.0,1.0,0.0,0.0,33.0,17431.0,63.0,2.0,2.0,4829.0,47.0,33800.0,0.0,1.0,1.0,2.0,2222.0,10551.0,52.0,0.0,0.0,77.0,199.0,3.0,3.0,0.0,3.0,NaN,12.0,NaN,1.0,3.0,3.0,4.0,7.0,6.0,13.0,18.0,3.0,14.0,0.0,0.0,0.0,2.0,95.7,33.3,1.0,0.0,61551.0,28880.0,22000.0,27751.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7000.0,232.44,40000.0,20.25,0.0,0.0,60.0,NaN,13.0,0.0,5004.0,29.0,0.0,7693.314943,0.0,60.0,NaN,NaN,0.0,0.0,131726.0,1.0,6.0,0.0,2.0,16.0,126722.0,102.0,2.0,2.0,3944.0,90.0,13900.0,2.0,1.0,4.0,4.0,10977.0,4996.0,50.0,0.0,0.0,122.0,132.0,1.0,1.0,0.0,10.0,64.0,5.0,60.0,3.0,2.0,2.0,3.0,4.0,19.0,7.0,10.0,2.0,13.0,0.0,0.0,0.0,2.0,89.7,33.3,0.0,0.0,132817.0,131726.0,10000.0,118917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000.0,326.92,52320.0,12.87,0.0,1.0,NaN,97.0,6.0,1.0,1692.0,26.0,0.0,11036.679274,0.0,NaN,NaN,NaN,0.0,0.0,74220.0,0.0,3.0,2.0,3.0,7.0,72528.0,113.0,1.0,1.0,1692.0,94.0,4400.0,2.0,1.0,4.0,4.0,12370.0,608.0,73.6,0.0,0.0,125.0,48.0,7.0,7.0,0.0,45.0,NaN,0.0,NaN,0.0,1.0,1.0,1.0,1.0,23.0,3.0,3.0,1.0,6.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,72124.0,74220.0,2300.0,67724.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28000.0,915.36,103000.0,30.76,0.0,0.0,73.0,NaN,12.0,0.0,21266.0,30.0,0.0,30922.184697,0.0,73.0,NaN,NaN,0.0,0.0,76475.0,3.0,3.0,2.0,3.0,4.0,55209.0,91.0,1.0,3.0,10750.0,41.0,80300.0,0.0,0.0,0.0,6.0,6373.0,59034.0,26.5,0.0,0.0,134.0,184.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,1.0,4.0,4.0,9.0,15.0,12.0,9.0,17.0,4.0,12.0,0.0,0.0,0.0,3.0,96.7,11.1,0.0,0.0,153679.0,76475.0,80300.0,73379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
num_cols_df.describe()

,loan_amnt,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
count,93853.000000,93853.000000,9.385300e+04,93608.000000,93853.000000,93853.000000,42283.000000,14067.000000,93853.000000,93853.000000,9.385300e+04,93853.000000,93853.000000,93853.000000,93853.00000,22997.000000,1.188400e+04,11884.000000,93853.000000,9.385300e+04,9.385300e+04,93853.000000,93853.000000,93853.000000,93853.000000,90727.000000,9.385300e+04,78660.000000,93853.000000,93853.000000,93853.000000,93825.000000,9.385300e+04,93853.000000,93853.000000,93853.000000,93853.000000,93843.000000,92324.000000,92256.000000,93853.000000,93853.000000,90727.000000,93853.000000,93853.000000,93853.00000,93853.000000,92430.000000,19460.000000,85724.000000,27916.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,91306.0,93853.000000,93853.000000,93853.000000,93852.000000,92322.000000,93853.000000,93853.000000,9.385300e+04,9.385300e+04,93853.000000,9.385300e+04,11884.000000,11884.000000,11884.000000,11884.000000,11667.000000,11884.000000,11884.000000,11884.000000,11884.000000,4101.000000
mean,15013.870627,449.354976,7.937007e+04,18.779895,0.235869,0.538928,36.955443,82.586834,11.412187,0.159739,1.442250e+04,23.844288,125.130223,13279.857748,0.01740,46.220246,1.255620e+05,18.932321,0.000064,3.258723e+02,1.511757e+05,1.055672,2.654119,0.846664,1.819132,18.150716,3.609237e+04,70.944317,1.372178,2.880004,5318.037005,52.883901,3.742583e+04,1.318690,1.713147,2.339499,5.035641,14763.011370,15898.812638,45.469279,0.008524,3.415213,121.242805,172.092805,13.670932,7.59313,1.495605,23.385935,40.723227,6.669334,37.939819,0.499750,3.324497,4.963304,4.754020,7.324145,8.750493,8.093902,13.393775,4.916337,11.386956,0.0,0.000064,0.063301,2.375928,94.694915,29.536810,0.144524,0.015066,1.919005e+05,5.073744e+04,25879.506526,4.542115e+04,33006.718445,0.710451,1.673847,11.285510,55.071235,2.907523,12.738219,0.048889,0.077415,37.376981
std,10021.202609,290.944311,7.208105e+04,20.558877,0.748662,0.790688,21.864641,23.666630,5.914010,0.444475,2.126069e+04,12.628098,1600.009813,10692.044200,0.14505,21.495350,6.899398e+04,8.204615,0.007995,2.660810e+04,1.696063e+05,1.213321,2.883313,1.039662,1.707400,23.317944,4.514236e+04,22.837397,1.573458,2.696749,5557.874003,21.976649,3.637919e+04,1.676191,2.924269,2.600783,3.447436,18258.463651,20421.735126,29.822890,0.109494,325.721023,54.657124,97.398013,17.206229,8.40579,1.805258,31.943845,22.179328,5.713125,22.140755,1.412741,2.301110,3.275671,3.183376,4.684539,7.632749,4.924931,8.188911,3.189622,5.906620,0.0,0.007995,0.417321,2.028524,8.827529,34.225894,0.358137,0.255500,1.889880e+05,5.214085e+04,25451.954725,4.717523e+04,28

### Non-numerical Columns

In [9]:
non_num_cols = df.select_dtypes(include=["object"])    # non-numerical columns
non_num_cols.columns 

Index(['term', 'int_rate', 'grade', 'emp_length', 'home_ownership',
       'loan_status', 'title', 'revol_util', 'application_type',
       'sec_app_earliest_cr_line'],
      dtype='object')

In [10]:
non_num_cols.head()

,term,int_rate,grade,emp_length,home_ownership,loan_status,title,revol_util,application_type,sec_app_earliest_cr_line
0,36 months,7.34%,A,9 years,RENT,Fully Paid,Major purchase,0.5%,Individual,NaN
1,36 months,11.98%,B,10+ years,OWN,Fully Paid,Other,33.9%,Individual,NaN
2,36 months,11.98%,B,< 1 year,MORTGAGE,Fully Paid,Home improvement,36%,Individual,NaN
3,36 months,10.90%,B,< 1 year,RENT,Fully Paid,Debt consolidation,38.5%,Individual,NaN
4,36 months,10.90%,B,6 years,MORTGAGE,Fully Paid,Debt consolidation,26.5%,Individual,NaN


In [11]:
non_num_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93853 entries, 0 to 93852
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   term                      93853 non-null  object
 1   int_rate                  93853 non-null  object
 2   grade                     93853 non-null  object
 3   emp_length                85734 non-null  object
 4   home_ownership            93853 non-null  object
 5   loan_status               93853 non-null  object
 6   title                     93853 non-null  object
 7   revol_util                93723 non-null  object
 8   application_type          93853 non-null  object
 9   sec_app_earliest_cr_line  11884 non-null  object
dtypes: object(10)
memory usage: 7.2+ MB


### Duplicates
Check for duplicates.

In [12]:
duplicates = df[df.duplicated()]    # check for duplicates
duplicates

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,issue_d,loan_status,title,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,total_pymnt,last_pymnt_d,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog


## Step 3: Change Data Type

#### int_rate and revol_util
- Change field's type from object to float
- Show ratio instead of percentage

In [13]:
df['int_rate'].head()

0      7.34%
1     11.98%
2     11.98%
3     10.90%
4     10.90%
Name: int_rate, dtype: object

In [14]:
df['revol_util'].head()

0     0.5%
1    33.9%
2      36%
3    38.5%
4    26.5%
Name: revol_util, dtype: object

In [15]:
# convert int_rate from percentage to ratio and from object to float
df['int_rate'] = df['int_rate'].apply(lambda x: float(x.strip('%'))/100)
df['revol_util'] = df['revol_util'].apply(lambda x: float(str(x).strip('%'))/100)

## Step 4: Handle Missing Values
- [Columns with more than 25% of missing values](#Columns-with-more-than-25%-of-missing-values)
- [Columns with 25% or less of missing values](#Columns-with-25%-or-less-of-missing-values)

Get columns that has missing values.

In [16]:
# compute null/non-null percentage
def compute_percentage(counts,flag):
    pct = 0
    if flag in counts.index:
        pct = round((counts[flag]/df.shape[0])*100, 2)
    return pct   
    
# create a dictionary of columns that contain null vs. non-null percentage
cols_dict = {}
for col in list(df.columns):
    counts = df[col].isnull().value_counts()         # count null values
    false_pct = compute_percentage(counts, False)    # compute non-null percentage
    true_pct = compute_percentage(counts, True)      # compute null percentage
    
    if false_pct < 100:  # only add column's name to dictionary if has missing values
        cols_dict[col] = {'non_null_pct': false_pct, 'null_pct': true_pct}

# create a data frame of columns that have missing values along with null vs. non-null percentage
cols_df = pd.DataFrame(cols_dict).T
cols_df.head()

,non_null_pct,null_pct
emp_length,91.35,8.65
dti,99.74,0.26
mths_since_last_delinq,45.05,54.95
mths_since_last_record,14.99,85.01
revol_util,99.86,0.14


### Columns with more than 25% of missing values

In [17]:
# get a list of columns that have > 25% missing values
high_null_counts = cols_df[cols_df.null_pct > 25]
high_null_counts

,non_null_pct,null_pct
mths_since_last_delinq,45.05,54.95
mths_since_last_record,14.99,85.01
mths_since_last_major_derog,24.50,75.50
annual_inc_joint,12.66,87.34
dti_joint,12.66,87.34
mths_since_recent_bc_dlq,20.73,79.27
mths_since_recent_revol_delinq,29.74,70.26
revol_bal_joint,12.66,87.34
sec_app_earliest_cr_line,12.66,87.34
sec_app_inq_last_6mths,12.66,87.34


In [18]:
# number of columns that have > 25% of missing values
high_null_counts.shape

(18, 2)

In [19]:
# drop columns that have > 25% missing values
df.drop(axis=1, columns=list(high_null_counts.index), inplace=True)
df.shape

(93853, 78)

### Columns with 25% or less of missing values
- [Non-numerical or Date Columns: Missing Values](#Non-numerical/Date-Columns:-Missing-Values)
- [Numerical Columns: Missing Values](#Numerical-Columns:-Missing-Values)

In [20]:
# get columns that have <= 25% missing values
low_null_counts = cols_df[cols_df.null_pct <= 25]
low_null_counts.shape

(15, 2)

In [21]:
low_null_counts

,non_null_pct,null_pct
emp_length,91.35,8.65
dti,99.74,0.26
revol_util,99.86,0.14
last_pymnt_d,99.32,0.68
mths_since_rcnt_il,96.67,3.33
il_util,83.81,16.19
all_util,99.97,0.03
avg_cur_bal,99.99,0.01
bc_open_to_buy,98.37,1.63
bc_util,98.30,1.70


##### Non-numerical or Date Columns: Missing Values
Leave the missing values as they are because the values may be missing for some reasons.

In [22]:
non_num_low_null_cols = list(df[list(low_null_counts.index)].select_dtypes(exclude=["number"]).columns)
low_null_counts.loc[non_num_low_null_cols]

,non_null_pct,null_pct
emp_length,91.35,8.65
last_pymnt_d,99.32,0.68


In [23]:
# view loan status for observation with missing value
for col in non_num_low_null_cols:
    print(col)
    print(df[df[col].isnull()]['loan_status'].value_counts())
    print('\n')

emp_length
Fully Paid     5912
Charged Off    2130
Default          77
Name: loan_status, dtype: int64


last_pymnt_d
Charged Off    636
Name: loan_status, dtype: int64




##### Numerical Columns: Missing Values

Numerical columns that have <= 25% missing values.

In [24]:
# get a list of numerical columns that have <= 25% missing values
num_low_null_cols = list(df[list(low_null_counts.index)].select_dtypes(include=["number"]).columns)
low_null_counts.loc[num_low_null_cols]

,non_null_pct,null_pct
dti,99.74,0.26
revol_util,99.86,0.14
mths_since_rcnt_il,96.67,3.33
il_util,83.81,16.19
all_util,99.97,0.03
avg_cur_bal,99.99,0.01
bc_open_to_buy,98.37,1.63
bc_util,98.30,1.70
mo_sin_old_il_acct,96.67,3.33
mths_since_recent_bc,98.48,1.52


Fill in missing values with the median.

In [25]:
for col in num_low_null_cols:
    df[col].fillna(df[col].median(), inplace=True)   # fill in missing values with the median

In [26]:
df[num_low_null_cols].describe()

,dti,revol_util,mths_since_rcnt_il,il_util,all_util,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mths_since_recent_bc,mths_since_recent_inq,num_tl_120dpd_2m,percent_bc_gt_75
count,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.000000,93853.0,93853.000000
mean,18.774935,0.400162,17.912544,71.438974,52.884234,14762.288217,15784.653463,45.423859,121.467870,23.228464,6.524746,0.0,29.327407
std,20.532254,0.254007,22.962199,20.937627,21.973379,18257.625297,20274.116998,29.570081,53.752839,31.726146,5.480251,0.0,33.984510
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000
25%,10.590000,0.194000,6.000000,62.000000,38.000000,3006.000000,3189.000000,20.300000,86.000000,6.000000,2.000000,0.0,0.000000
50%,16.880000,0.372000,11.000000,74.000000,54.000000,7976.000000,8891.500000,42.800000,128.000000,13.000000,5.000000,0.0,16.700000
75%,24.130000,0.585000,20.000000,84.000000,68.000000,20743.000000,20399.000000,69.700000,152.000000,27.000000,9.000000,0.0,50.000000
max,999.000000,1.322000,383.000000,309.000000,175.000000,513930.000000,371701.000000,158.600000,822.000000,551.000000,25.000000,0.0,100.000000


## Step 5: Handle Outliers

Compute z-score for each numerical columns.
If the column contains outliers, store the column's name and the outlier records

In [27]:
from scipy.stats import zscore
import numpy as np
x = np.seterr(divide='ignore', invalid='ignore')

# get a list of numerical columns
num_cols = list(df.select_dtypes(include=['number']).columns)

outlier_cols = []     # columns that have outliers
outliers_list = []

# get number of outliers for each numerical columns
for col in num_cols:
    z_score = np.abs(zscore(df[col]))
    if len(z_score) == 0:
        continue
    
    outliers = (z_score < -3) | (z_score > 3)  # construct a Boolean Series to identify outliers
    if(df[outliers].shape[0] > 0):             # if there are outliers
        outlier_cols.append(col)               # store column's name
        outliers_list.append(df[outliers])     # store data frame that contains outliers

# combine outlier data frames and drop duplicates
outliers_df = pd.concat(outliers_list).drop_duplicates()  

In [28]:
# number of rows and columns that contain extreme values
outliers_df[outlier_cols].shape

(36355, 64)

In [29]:
df.shape

(93853, 78)

In [30]:
# list of columns that contain outliers
print(outlier_cols)

['int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'total_pymnt', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pub_rec_bankruptcies', 'tax_

## Step 6: Add New Columns

### Loan Status Flag
Categorize loan status into 2 categories:
- Fully Paid
- Default (Default, Charged Off)

In [31]:
df['loan_status'].value_counts()

Fully Paid     76264
Charged Off    16803
Default          786
Name: loan_status, dtype: int64

In [32]:
# add new column based on the value of loan_status
def set_loan_flag(status):
    for index, status_list in enumerate(gv.LOAN_STATUS):
        if status in status_list:
            return gv.LOAN_STATUS_FLAG[index]

df['loan_status_flag'] = df['loan_status'].apply(set_loan_flag)

# drop column 'loan_status'
df.drop(['loan_status'], axis=1, inplace=True)

In [33]:
df['loan_status_flag'].value_counts()

Fully Paid    76264
Default       17589
Name: loan_status_flag, dtype: int64

### Return on Investment (ROI)

In [34]:
# roi = total loan payment - loan amount
df['roi'] = df['total_pymnt'] - df['loan_amnt'] 

### Months in Loan

In [35]:
# months in loan = issue date - last payment date
df['months_in_loan'] = round((df['last_pymnt_d'] - df['issue_d']) / np.timedelta64(1,'M'))

## Step 7: Change Object Column Type to Category

In [36]:
excluded_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d']

In [37]:
cat_cols = set(list(df.select_dtypes(exclude=['number']).columns)) - set(excluded_cols)

In [38]:
cat_cols

{'application_type',
 'emp_length',
 'grade',
 'home_ownership',
 'loan_status_flag',
 'term',
 'title'}

In [39]:
# convert to categories
for col in cat_cols:
    if col == 'emp_length':
        ordered_cat = pd.api.types.CategoricalDtype(categories=['< 1 year', '1 year', '2 years', '3 years', 
                                                                '4 years', '5 years', '6 years', '7 years', 
                                                                '8 years', '9 years', '10+ years'], ordered=True)
    elif col == 'loan_status_flag':
        ordered_cat = pd.api.types.CategoricalDtype(categories=gv.LOAN_STATUS_FLAG, ordered=True)
    else:
        ordered_cat = pd.api.types.CategoricalDtype(categories=list(df[col].value_counts().sort_index().index),
                                                   ordered=True)
    df[col] = df[col].astype(ordered_cat)

## Step 8: Export Data

In [40]:
# remove leading and trailing spaces for all category columns
for col_name in list(df.select_dtypes(include=["category"]).columns):  
    df[col_name] = df[col_name].str.strip()

In [41]:
#df.to_csv('out/2018_LC_Loans_Cleaned.csv', index=False)   # csv file
df.to_pickle('out/2018_LC_Loans_Cleaned.pkl')             # pickle file

In [42]:
# build data dictionary for variables of interest
data_dict = pd.read_excel('data/LCDataDictionary.xlsx')
data_dict = data_dict[data_dict['LoanStatNew'].isin(list(df.columns))]
data_dict.to_csv('data/data_dictionary.csv')